In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import kraft

In [4]:
di = "../raw/"

## Check raw

In [6]:
na_pa = {}

for pa in kraft.path.get_child(di):

    if pa[-3:] != ".md":

        continue

    na = pa.split("/")[-1][: -len(".md")]

    assert na == kraft.string.untitle(na), na

    na_pa[na] = pa

    se_ = [".", "<", ">"]

    with open(pa) as io:

        for ie, li in enumerate(io):

            li = li.strip()

            er = "{}: {}".format(na, li)

            if ie % 2 == 1:

                assert li == "", er

            elif li[0] == "#":

                se = li[2]

                assert se == se_.pop(0), er

            elif se == ".":

                assert li[-1] == ".", er

len(na_pa)

382

## Read raw

In [7]:
na_se_li_ = {}

for na, pa in na_pa.items():

    se_li_ = {
        ".": [],
        "<": [],
        ">": [],
    }

    with open(pa) as io:

        for li in io.readlines()[0::2]:

            li = li.strip()

            if li[0] == "#":

                se = li[2]

            else:

                se_li_[se].append(li)

    na_se_li_[na] = se_li_

## Link .md

In [8]:
for se_li_ in na_se_li_.values():

    se_li_["md<"] = []

    se_li_[">md"] = []

for na, se_li_ in na_se_li_.items():

    for li in list(se_li_[">"]):

        li2 = kraft.string.untitle(li)

        if li2 in na_se_li_:

            se_li_[">"].remove(li)

            se_li_[">md"].append(li2)

            na_se_li_[li2]["md<"].append(na)

## Check collision

In [9]:
def combine_future(se_li_):

    return se_li_[">"] + kraft.iterable.flatten(
        [na_se_li_[li][">"] for li in se_li_[">md"]]
    )

In [10]:
import pandas as pd

In [11]:
for na, se_li_ in na_se_li_.items():

    for li, n_re in pd.value_counts(combine_future(se_li_)).items():

        if 1 < n_re:

            so_ = [na]

            so_ += [li2 for li2 in se_li_[">md"] if li in na_se_li_[li2][">"]]

            # print("{}:\n\t{}\n".format(li, "\n\t".join(so_)))

## Read feature

In [12]:
fe_pr = pd.read_csv(
    "../feature.tsv", "\t", index_col=0, squeeze=True, dtype=str
).to_dict()

## Check extra feature 

In [13]:
fe_ = []

for se_li_ in na_se_li_.values():

    fe_ += se_li_["<"] + se_li_[">"]

In [14]:
ex_ = []

for fe in fe_pr.keys():

    if fe not in fe_:

        ex_.append(fe)

assert len(ex_) == 0, ex_

AssertionError: ['Myalgia', 'Panic attack', 'Kidney disease', 'Bradycardia', 'Palpitation']

## Check missing feature

In [ ]:
mi_ = []

for na, se_li_ in na_se_li_.items():

    for li in se_li_["<"] + se_li_[">"]:

        if li not in fe_pr:

            mi_.append(li)

assert len(mi_) == 0, "\n" + "\n".join(set(mi_))

## Rename feature

In [ ]:
def number(pr):

    return [int(it) for it in pr.split(".")]

In [ ]:
def order(se):

    return [number(pr) + [fe] for fe, pr in se.items()]

In [ ]:
se = pd.Series(fe_pr).sort_values(key=order)

se.index.name = "Feature"

se.name = "Priority"

se.to_csv("../feature.tsv", "\t")

## Rewrite

In [ ]:
def prioritize(fe):

    if fe.endswith(".") or kraft.string.untitle(fe) in na_se_li_:

        return number("0.0.0") + [fe]

    return number(fe_pr[fe]) + [fe]

In [ ]:
def sort(li_):

    return sorted(li_, key=prioritize)

In [ ]:
for na, se_li_ in na_se_li_.items():

    with open(na_pa[na], "w") as io:

        ti_li_ = {
            ".": se_li_["."],
            "<": se_li_["<"],
            ">": se_li_[">"] + [kraft.string.title(li).upper() for li in se_li_[">md"]],
        }

        for ti, li_ in ti_li_.items():

            if ti != ".":

                io.write("\n")

            io.write("# {}\n".format(ti))

            if 0 < len(li_):

                io.write("\n")

                io.write("\n\n".join(sort(li_)) + "\n")

## Clean

In [ ]:
import os
import shutil

In [ ]:
di2 = "../clean/"

if os.path.isdir(di2):

    shutil.rmtree(di2)

In [ ]:
for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace(di, di2)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            "Comment": se_li_["."],
            "Epidemiology": [],
            "Cause": [kraft.string.title(li) for li in se_li_["md<"]],
            "Symptom": [],
            "Sign": [],
            "Diagnostic": [],
            # "Treatment and Management": [],
        }

        for li in se_li_["<"]:

            nu_ = number(fe_pr[li])

            if [2, 0, 0] <= nu_ < [6, 0, 0]:

                ti_li_["Epidemiology"].append(li)

        for li in combine_future(se_li_):

            nu_ = number(fe_pr[li])

            if [1, 0, 0] <= nu_ < [2, 0, 0]:

                ti_li_["Symptom"].append(li)

            elif [6, 0, 0] <= nu_ < [7, 0, 0]:

                ti_li_["Sign"].append(li)

            elif [7, 0, 0] <= nu_:

                ti_li_["Diagnostic"].append(li)

        for ti, li_ in ti_li_.items():

            if 0 < len(li_):

                if ti != "Comment":

                    io.write("\n")

                io.write("# {}\n".format(ti))

                io.write("\n")

                io.write("\n\n".join(sort(li_)) + "\n")

## Plan physical exam